# Data Collection
## Pull data from Yahoo Finance

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

In [2]:
# Define ticker symbols and date range
tickers = [
    "AAPL",  # Apple
    "MSFT",  # Microsoft
    "NVDA",  # Nvidia
    "SAP",   # SAP
    "TSM"    # Taiwan Semi
]

# Set start date to January 1, 2015
start_date = '2015-01-01'

# Set end date to today's date
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

In [3]:
# Download price data for all tickers
def price_data(tickers, start_date, end_date):
    price_dfs = []

    for ticker in tickers:
        print(f"Downloading data for {ticker}...")

        # Download data from Yahoo Finance
        df = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=False)

        # Flatten MultiIndex columns if Yahoo returns one
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = [c[0] for c in df.columns]

       # Keep ALL OHLCV columns
        needed_cols = ["Open", "High", "Low", "Close", "Adj Close", "Volume"]

        # Some foreign tickers occasionally miss a column — safety check
        for col in needed_cols:
            if col not in df.columns:
                df[col] = None

        # Keep only the needed columns
        df = df[needed_cols].reset_index()

        # Add ticker
        df["Ticker"] = ticker

        # Reorder columns
        df = df[["Date", "Ticker", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]

        price_dfs.append(df)

    # Combine into one dataset
    all_prices = pd.concat(price_dfs, ignore_index=True)
    return all_prices

# Get price data
price_data = price_data(tickers, start_date, end_date)
price_data.sample(2)


,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
6139,2017-09-07,NVDA,4.162250,4.197250,4.123750,4.164500,4.115729,352868000
11752,2018-04-12,TSM,43.330002,43.849998,43.049999,43.849998,36.281254,7574700


In [4]:
# Download fundmanental data for all tickers
def fundamental_data(tickers):
    fundamentals_list = []

    # Fields we want to extract for each company
    fields = [
        "longName",
        "sector",
        "industry",
        "marketCap",
        "enterpriseValue",
        "trailingPE",
        "forwardPE",
        "priceToBook",
        "bookValue",
        "trailingEps",
        "returnOnEquity",
        "returnOnAssets",
        "profitMargins",
        "operatingMargins",
        "grossProfits",
        "freeCashflow",
        "revenueGrowth",
        "earningsGrowth",
        "revenueQuarterlyGrowth",
        "debtToEquity",
        "currentRatio",
        "quickRatio",
        "beta",
        "dividendYield",
        "targetMeanPrice"
    ]

    for ticker in tickers:
        print(f"Fetching fundamentals for {ticker}...")

        stock = yf.Ticker(ticker)

        try:
            info = stock.info
        except Exception as e:
            print(f"⚠️ Could not fetch data for {ticker}: {e}")
            continue

        # Build one clean row for this ticker
        row = {"Ticker": ticker}

        for f in fields:
            row[f] = info.get(f, None)

        fundamentals_list.append(row)

    # Convert list of dicts to DataFrame
    fundamentals_df = pd.DataFrame(fundamentals_list)

    return fundamentals_df

# Get fundamental data
fundamental_data = fundamental_data(tickers)
fundamental_data.sample(2)


Fetching fundamentals for AAPL...
Fetching fundamentals for MSFT...
Fetching fundamentals for NVDA...
Fetching fundamentals for SAP...
Fetching fundamentals for TSM...


,Ticker,longName,sector,industry,marketCap,enterpriseValue,trailingPE,forwardPE,priceToBook,bookValue,...,freeCashflow,revenueGrowth,earningsGrowth,revenueQuarterlyGrowth,debtToEquity,currentRatio,quickRatio,beta,dividendYield,targetMeanPrice
4,TSM,Taiwan Semiconductor Manufacturing Company Lim...,Technology,Semiconductors,1507307290624,5792912113664,32.291110,35.967823,1.507662,192.762,...,638052335616,0.303,0.391,None,18.998,2.693,2.397,1.260,1.15,339.90668
3,SAP,SAP SE,Technology,Software - Application,295040843776,295719305216,36.347202,37.531853,6.888731,36.776,...,7024249856,0.072,0.379,None,21.093,1.105,0.943,0.813,1.00,339.83334


In [5]:
# Create technical indicators for all tickers
def add_technical_indicators(price_df):
    df_list = []

    for ticker in price_df["Ticker"].unique():
        tdf = price_df[price_df["Ticker"] == ticker].copy()
        tdf = tdf.sort_values("Date")

        # Return 1 day
        tdf["Return_1d"] = tdf["Close"].pct_change()

        # Moving Averages
        tdf["MA10"] = tdf["Close"].rolling(10).mean()
        tdf["MA20"] = tdf["Close"].rolling(20).mean()
        tdf["MA50"] = tdf["Close"].rolling(50).mean()

        # Volatility
        tdf["Volatility20"] = tdf["Return_1d"].rolling(20).std()

        # RSI
        delta = tdf["Close"].diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)
        avg_gain = gain.rolling(14).mean()
        avg_loss = loss.rolling(14).mean()
        rs = avg_gain / (avg_loss + 1e-10)
        tdf["RSI14"] = 100 - (100 / (1 + rs))

        # MACD
        ema12 = tdf["Close"].ewm(span=12, adjust=False).mean()
        ema26 = tdf["Close"].ewm(span=26, adjust=False).mean()
        macd = ema12 - ema26
        signal = macd.ewm(span=9, adjust=False).mean()

        tdf["MACD"] = macd
        tdf["MACD_Signal"] = signal

        # Bollinger Bands
        mb = tdf["Close"].rolling(20).mean()
        std = tdf["Close"].rolling(20).std()

        tdf["BB_Middle"] = mb
        tdf["BB_Upper"] = mb + 2 * std
        tdf["BB_Lower"] = mb - 2 * std

        # OBV
        tdf["OBV"] = (np.sign(tdf["Close"].diff()) * tdf["Volume"]).fillna(0).cumsum()

        df_list.append(tdf)

    final_df = pd.concat(df_list, ignore_index=True)
    return final_df


# Get technical indicators
indicators = add_technical_indicators(price_data)
indicators.sample(2)

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Return_1d,MA10,MA20,MA50,Volatility20,RSI14,MACD,MACD_Signal,BB_Middle,BB_Upper,BB_Lower,OBV
13195,2024-01-05,TSM,99.000000,100.639999,98.800003,99.610001,97.175346,7347300,0.004842,102.393999,102.054,97.585,0.014262,38.557817,0.818019,1.537976,102.054,105.762922,98.345078,565190200.0
11568,2017-07-19,TSM,36.060001,36.419998,36.060001,36.410000,30.125443,7033800,0.015054,35.579000,35.599,35.619,0.011611,63.874378,0.143296,0.034015,35.599,37.094437,34.103563,284140900.0


In [6]:
# Create target variable for all tickers
def create_target_variable(df, horizon=5):
    df = df.sort_values(["Ticker", "Date"]).reset_index(drop=True)

    # Create a new column: future close price shifted by horizon
    df["Close_Future"] = df.groupby("Ticker")["Close"].shift(-horizon)

    # Calculate % return
    df["Target_Return"] = (df["Close_Future"] - df["Close"]) / df["Close"]

    # Optionally make classification (1 = up, 0 = down)
    df["Target_UpDown"] = (df["Target_Return"] > 0).astype(int)

    # Drop rows where the target cannot be computed (last horizon days)
    df = df.dropna(subset=["Target_Return"])

    return df

# Create target variable
data = create_target_variable(indicators, horizon=5)
data.sample(2)

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Return_1d,MA10,...,RSI14,MACD,MACD_Signal,BB_Middle,BB_Upper,BB_Lower,OBV,Close_Future,Target_Return,Target_UpDown
6067,2017-05-25,NVDA,3.500000,3.500750,3.411000,3.456500,3.413016,608228000,-0.002237,3.371400,...,79.093380,0.229643,0.191281,3.03555,3.798180,2.27292,3.518411e+10,3.591000,0.038912,1
11681,2017-12-28,TSM,39.509998,39.889999,39.400002,39.740002,32.880669,4279500,0.016888,39.371001,...,62.126324,-0.357706,-0.431753,39.31250,39.853061,38.77194,3.744824e+08,42.459999,0.068445,1


In [7]:
# Merge data and fundamental data
data = data.merge(fundamental_data, on="Ticker", how="left")
data.sample(2)

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Return_1d,MA10,...,freeCashflow,revenueGrowth,earningsGrowth,revenueQuarterlyGrowth,debtToEquity,currentRatio,quickRatio,beta,dividendYield,targetMeanPrice
6990,2021-02-09,NVDA,14.340750,14.583250,14.222500,14.263250,14.224043,287560000,-0.012155,13.524,...,52436750336,0.556,0.612,None,10.584,4.214,3.488,2.269,0.02,230.77684
12006,2019-05-15,TSM,41.549999,42.330002,41.459999,41.970001,35.995804,5347700,0.000000,43.134,...,638052335616,0.303,0.391,None,18.998,2.693,2.397,1.260,1.15,339.90668


In [8]:
# Columns
data.columns

Index(['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Return_1d', 'MA10', 'MA20', 'MA50', 'Volatility20', 'RSI14', 'MACD',
       'MACD_Signal', 'BB_Middle', 'BB_Upper', 'BB_Lower', 'OBV',
       'Close_Future', 'Target_Return', 'Target_UpDown', 'longName', 'sector',
       'industry', 'marketCap', 'enterpriseValue', 'trailingPE', 'forwardPE',
       'priceToBook', 'bookValue', 'trailingEps', 'returnOnEquity',
       'returnOnAssets', 'profitMargins', 'operatingMargins', 'grossProfits',
       'freeCashflow', 'revenueGrowth', 'earningsGrowth',
       'revenueQuarterlyGrowth', 'debtToEquity', 'currentRatio', 'quickRatio',
       'beta', 'dividendYield', 'targetMeanPrice'],
      dtype='object')

In [10]:
# Check for missing values
data.isnull().sum()

Date                          0
Ticker                        0
Open                          0
High                          0
Low                           0
Close                         0
Adj Close                     0
Volume                        0
Return_1d                     5
MA10                         45
MA20                         95
MA50                        245
Volatility20                100
RSI14                        70
MACD                          0
MACD_Signal                   0
BB_Middle                    95
BB_Upper                     95
BB_Lower                     95
OBV                           0
Close_Future                  0
Target_Return                 0
Target_UpDown                 0
longName                      0
sector                        0
industry                      0
marketCap                     0
enterpriseValue               0
trailingPE                    0
forwardPE                     0
priceToBook                   0
bookValu

In [9]:
# Save datasets to the Data folder
data.to_csv("../Data/Stock_Data.csv", index=False)